In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

from api_keys import google, openweather

In [8]:
file = 'weather.csv'
df=pd.read_csv(file)

print(df.columns)

Index(['Unnamed: 0', 'weather', 'base', 'visibility', 'dt', 'timezone', 'id',
       'name', 'cod', 'coord.lon', 'coord.lat', 'main.temp', 'main.feels_like',
       'main.temp_min', 'main.temp_max', 'main.pressure', 'main.humidity',
       'main.sea_level', 'main.grnd_level', 'wind.speed', 'wind.deg',
       'wind.gust', 'clouds.all', 'sys.country', 'sys.sunrise', 'sys.sunset',
       'sys.type', 'sys.id', 'message', 'rain.1h', 'snow.1h'],
      dtype='object')


In [6]:
gmaps.configure(api_key=google)

In [9]:
locations = df[["coord.lat", "coord.lon"]].astype(float)

humidity = df["main.humidity"].astype(float)

In [15]:
fig = gmaps.figure()


heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [26]:
hotel_df = df.loc[(df["main.temp"] <85), :]
hotel_df = hotel_df.loc[(df["main.temp"] >75), :]
hotel_df= hotel_df.loc[(df["wind.speed"] <10), :]
hotel_df= hotel_df.loc[(df["wind.gust"] <4), :]
hotel_df= hotel_df.loc[(df["clouds.all"] <5), :]
hotel_df

,Unnamed: 0,weather,base,visibility,dt,timezone,id,name,cod,coord.lon,...,wind.gust,clouds.all,sys.country,sys.sunrise,sys.sunset,sys.type,sys.id,message,rain.1h,snow.1h
119,0,"[{'id': 800, 'main': 'Clear', 'description': '...",stations,10000.0,1.620917e+09,28800.0,2062276.0,Roebourne,200,117.1333,...,2.77,0.0,AU,1.620859e+09,1.620899e+09,NaN,NaN,NaN,NaN,NaN
223,0,"[{'id': 800, 'main': 'Clear', 'description': '...",stations,10000.0,1.620917e+09,28800.0,1720223.0,Callaguip,200,120.4867,...,3.06,4.0,PH,1.620855e+09,1.620901e+09,NaN,NaN,NaN,NaN,NaN
304,0,"[{'id': 800, 'main': 'Clear', 'description': '...",stations,10000.0,1.620917e+09,7200.0,214389.0,Kaniama,200,24.1833,...,1.01,0.0,CD,1.620880e+09,1.620922e+09,NaN,NaN,NaN,NaN,NaN
446,0,"[{'id': 800, 'main': 'Clear', 'description': '...",stations,10000.0,1.620917e+09,28800.0,6620339.0,Karratha,200,116.8463,...,2.55,0.0,AU,1.620859e+09,1.620899e+09,NaN,NaN,NaN,NaN,NaN
